In [1]:
import numpy as np 
import pandas as pd 
from scipy import signal
import matplotlib.pyplot as plt

In [2]:
#Load data
Walk_1_1 = pd.read_csv("Walk_1_Test/Walk_1_1.csv", delim_whitespace=True, header=None)
Walk_1_2 = pd.read_csv("Walk_1_Test/Walk_1_2.csv", delim_whitespace=True, header=None)
Walk_1_3 = pd.read_csv("Walk_1_Test/Walk_1_3.csv", delim_whitespace=True, header=None)
Walk_1_4 = pd.read_csv("Walk_1_Test/Walk_1_4.csv", delim_whitespace=True, header=None)
Walk_1_5 = pd.read_csv("Walk_1_Test/Walk_1_5.csv", delim_whitespace=True, header=None)
Walk_1_6 = pd.read_csv("Walk_1_Test/Walk_1_6.csv", delim_whitespace=True, header=None)
Walk_1_7 = pd.read_csv("Walk_1_Test/Walk_1_7.csv", delim_whitespace=True, header=None)
Walk_1_8 = pd.read_csv("Walk_1_Test/Walk_1_8.csv", delim_whitespace=True, header=None)
Walk_1_9 = pd.read_csv("Walk_1_Test/Walk_1_9.csv", delim_whitespace=True, header=None) 
Walk_1_10 = pd.read_csv("Walk_1_Test/Walk_1_10.csv", delim_whitespace=True, header=None)

Walk_1_1[7]=1
Walk_1_2[7]=2
Walk_1_3[7]=3
Walk_1_4[7]=4
Walk_1_5[7]=5                  
Walk_1_6[7]=6
Walk_1_7[7]=7
Walk_1_8[7]=8
Walk_1_9[7]=9
Walk_1_10[7]=10
          
agro=[Walk_1_1, Walk_1_2, Walk_1_3, Walk_1_4, Walk_1_5, Walk_1_6, 
      Walk_1_7, Walk_1_8, Walk_1_9, Walk_1_10]          
          
Walk_1=pd.concat(agro)  

x_mean=[]
y_mean=[]


In [3]:
#Functions
def plot_accel(x,y_xx,y_yy,y_zz): 
    plt.subplot(3, 1, 1)
    plt.plot(x, y_xx)
    #plt.title('Walking Accelerations')
    plt.ylabel('X accel')
    plt.subplot(3, 1, 2)
    plt.plot(x, y_yy)
    plt.ylabel('Y accel')
    plt.subplot(3, 1, 3)
    plt.plot(x, y_zz)
    plt.ylabel('Z accel')
    plt.xlabel('Time (s)')
    plt.show()

def plot_gyro(x,y_yaw,y_pitch,y_roll):
    plt.subplot(3, 1, 1)
    plt.plot(x, y_yaw)
    #plt.title('Walking Gyroscopic Data')
    plt.ylabel('Yaw')
    plt.subplot(3, 1, 2)
    plt.plot(x, y_pitch)
    plt.ylabel('Pitch')
    plt.subplot(3, 1, 3)
    plt.plot(x, y_roll)
    plt.ylabel('Roll')
    plt.xlabel('Time (s)')
    plt.show()

In [4]:
#accumulate data for correlation
yaw_accum=np.empty((10,200))
r_accum=np.empty((10,200))
p_accum=np.empty((10,200))
x_accum=np.empty((10,200))
y_accum=np.empty((10,200))
z_accum=np.empty((10,200))

for i in range(1,11):
    data=Walk_1[Walk_1[7]==i]
    x=-data[0] 
    
    yaw=data[1]
    y_yaw=signal.medfilt(data[1],15)
    if i==1:
        yaw_base=y_yaw[35]
    else:
        y_yaw=y_yaw-(y_yaw[35]-yaw_base)    
    
    
    #y_yaw=y_yaw-np.mean(y_yaw)
    #y_yaw=y_yaw[yaw]
    #shift down data so inital point the same
    #if i==1:
        #yaw_base=y_yaw.mean()
    #else:
        #y_yaw=y_yaw-(y_yaw.mean()-yaw_base)    
    x_yaw=x[yaw]

    #filters
    pitch=signal.medfilt(data[2])
    y_pitch=signal.medfilt(data[2])
    y_pitch=y_pitch-np.mean(y_pitch)
    x_pitch=x[pitch]

    roll=signal.medfilt(data[3])
    y_roll=signal.medfilt(data[3])
    y_roll=y_roll-np.mean(y_roll)        
    x_roll=x[roll]

    xx=signal.medfilt(data[4]) #filter based on value
    y_xx=signal.medfilt(data[4]) 
    y_xx=y_xx-np.mean(y_xx)     
    x_xx=x[xx]

    yy=signal.medfilt(data[5])
    y_yy=signal.medfilt(data[5])
    y_yy=y_yy-np.mean(y_yy)       
    x_yy=x[yy]
    
    zz=signal.medfilt(data[6])
    y_zz=signal.medfilt(data[6]) 
    y_zz=y_zz-np.mean(y_zz)        
    x_zz=x[zz]
    
    
    #plot_accel(x,y_xx, y_yy,y_zz)
    plot_gyro(x,y_yaw, y_pitch,y_roll)
    
    
    #record first 200 points    
    yaw_accum[i-1]=y_yaw[:200]
    r_accum[i-1]=y_roll[:200]
    p_accum[i-1]=y_pitch[:200]
    x_accum[i-1]=y_xx[:200]
    y_accum[i-1]=y_yy[:200]
    z_accum[i-1]=y_zz[:200]

In [5]:
#to see which is shorter
#problem=[]
#diff=np.zeros(10)

#for analyzing max accelerations
#zz2=np.empty((11,358))

#Generate correlation coefficient matricies
corr_yaw=np.corrcoef([yaw_accum[0],yaw_accum[1],yaw_accum[2],yaw_accum[3],yaw_accum[4],\
                yaw_accum[5], yaw_accum[6],yaw_accum[7],yaw_accum[8],yaw_accum[9]])

corr_r=np.corrcoef([r_accum[0],r_accum[1],r_accum[2],r_accum[3],r_accum[4],\
                r_accum[5], r_accum[6],r_accum[7],r_accum[8],r_accum[9]])
    
corr_p=np.corrcoef([p_accum[0],p_accum[1],p_accum[2],p_accum[3],p_accum[4],\
                p_accum[5], p_accum[6],p_accum[7],p_accum[8],p_accum[9]])

corr_x=np.corrcoef([x_accum[0],x_accum[1],x_accum[2],x_accum[3],x_accum[4],\
                x_accum[5], x_accum[6],x_accum[7],x_accum[8],x_accum[9]])
    
corr_y=np.corrcoef([y_accum[0],y_accum[1],y_accum[2],y_accum[3],y_accum[4],\
                y_accum[5], y_accum[6],y_accum[7],y_accum[8],y_accum[9]])
    
corr_z=np.corrcoef([z_accum[0],z_accum[1],z_accum[2],z_accum[3],z_accum[4],\
                z_accum[5], z_accum[6],z_accum[7],z_accum[8],z_accum[9]])   

#find average value of correlations    
avg_cor_yaw=np.mean(corr_yaw)     
avg_cor_roll=np.mean(corr_r)    
avg_cor_pitch=np.mean(corr_p) 
avg_cor_x=np.mean(corr_x)     
avg_cor_y=np.mean(corr_y)    
avg_cor_z=np.mean(corr_z)     
    
#build table of correlfation coeefs

#find average value of correlations (removing diagonal==1)    
avg_cor_yaw=np.mean(corr_yaw[corr_yaw!=1])     
avg_cor_roll=np.mean(corr_r[corr_r!=1])    
avg_cor_pitch=np.mean(corr_p[corr_p!=1]) 
avg_cor_x=np.mean(corr_x[corr_x!=1])     
avg_cor_y=np.mean(corr_y[corr_y!=1])    
avg_cor_z=np.mean(corr_z[corr_z!=1])     

types=['Yaw','Roll', 'Pitch','X acceleration', 'Y acceleration', 'Z acceleration']    
alpha=['Average Correlation Coefficient', 'Standard Deviation Correlation Coefficient']
averages_walk=pd.DataFrame(index=types, columns=[alpha])    
averages_walk.iloc[0]=[avg_cor_yaw, np.std(corr_yaw[corr_yaw!=1])]
averages_walk.iloc[1]=[avg_cor_roll, np.std(corr_r[corr_r!=1])]
averages_walk.iloc[2]=[avg_cor_pitch, np.std(corr_p[corr_p!=1])]
averages_walk.iloc[3]=[avg_cor_x, np.std(corr_x[corr_x!=1])]
averages_walk.iloc[4]=[avg_cor_y, np.std(corr_y[corr_y!=1])]
averages_walk.iloc[5]=[avg_cor_z, np.std(corr_z[corr_z!=1])]


averages_walk.to_csv("averages_walk.csv")



   
"""
    #xx=np.abs(np.diff(xx))<.5 #filter bases on derivative
    #xx=np.insert(xx, 0, True) #add first point as has no derivative
    xx=np.abs(np.diff(np.diff(data[4])))<2
    xx=np.insert(xx, 0, True)
    xx=np.insert(xx, 0, True)
    
    
    #yy=np.abs(np.diff(np.diff(data[5])))<.6
    #yy=np.insert(yy, 0, True)
    
    #zz2[i]=np.sort(np.abs(np.diff(np.diff(data[6]))))
    #zz=np.abs(np.diff(np.diff(data[6])))<200
    zz=data[6]
    accel=np.abs(np.diff(np.diff(zz)))
    i=1
    while i<=(len(zz)-3):
        if accel[i]>2:
            print(i)
            zz=zz.drop(i+1)
            accel=np.abs(np.diff(np.diff(zz)))
        else:
            i+=1
            print(i)
       #i+=1
    y_zz=data[6][zz.index]        
    x_zz=x[zz.index]
"""


'\n    #xx=np.abs(np.diff(xx))<.5 #filter bases on derivative\n    #xx=np.insert(xx, 0, True) #add first point as has no derivative\n    xx=np.abs(np.diff(np.diff(data[4])))<2\n    xx=np.insert(xx, 0, True)\n    xx=np.insert(xx, 0, True)\n    \n    \n    #yy=np.abs(np.diff(np.diff(data[5])))<.6\n    #yy=np.insert(yy, 0, True)\n    \n    #zz2[i]=np.sort(np.abs(np.diff(np.diff(data[6]))))\n    #zz=np.abs(np.diff(np.diff(data[6])))<200\n    zz=data[6]\n    accel=np.abs(np.diff(np.diff(zz)))\n    i=1\n    while i<=(len(zz)-3):\n        if accel[i]>2:\n            print(i)\n            zz=zz.drop(i+1)\n            accel=np.abs(np.diff(np.diff(zz)))\n        else:\n            i+=1\n            print(i)\n       #i+=1\n    y_zz=data[6][zz.index]        \n    x_zz=x[zz.index]\n'